In [1]:
# Download the current WALS data set to the cwd:
download.file("https://cdstar.shh.mpg.de/bitstreams/EAEA0-7269-77E5-3E10-0/wals_dataset.cldf.zip",
              destfile = "wals_dataset.cldf.zip")

# Read language and value information from the ZIP file:
languages <- read.csv(unz("wals_dataset.cldf.zip", "languages.csv"),
                      header=TRUE, sep=",")
values <- read.csv(unz("wals_dataset.cldf.zip", "values.csv"),
                   header=TRUE, sep=",")

In [2]:
# Let's lookup the feature "Hand and Arm" that has got two levels:
# Identical and Different, for the same or different lexemes for hand/arm
# in a language, respectively:
feature <- "129A"
values.filtered <- values[values$Parameter_ID == feature,]
values.filtered <- droplevels(values.filtered)

# Naturally, the amount of languages we have information
# concerning this feature is identical to the information available
# in WALS online:
dim(values.filtered)[1] == (228 + 389)

[1] TRUE

In [3]:
# Make a simple barplot:
barplot(table(values.filtered$Value), ylim = c(0, 400))

In [4]:
# We want some more information about the languages, so let's
# merge the filtered list from above with a list containing language
# information:
languages.filtered <- languages[languages$ID %in% values.filtered$Language_ID,]
colnames(languages.filtered)[1] <- "Language_ID"
merged <- merge(languages.filtered, values[values$Parameter_ID == feature,], by = "Language_ID", all = TRUE)

# Merged now has our feature information list as well as language information, for example
# geographical coordinates for the languages. Let's build a (simple, non-sensical) regression
# model and see whether hand/arm-colexification is connected to spatial information:
glm.analysis <- glm(merged$Value ~ merged$Longitude + merged$Latitude, family = "binomial")

# Having eyeballed the map for about five seconds there is indeed a strong confirmation
# in our model for a specific geographic distribution:
summary(glm.analysis)


Call:
glm(formula = merged$Value ~ merged$Longitude + merged$Latitude, 
    family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4551  -0.9608  -0.7946   1.2146   1.9447  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -0.6903739  0.1005661  -6.865 6.65e-12 ***
merged$Longitude  0.0045997  0.0009496   4.844 1.27e-06 ***
merged$Latitude   0.0133588  0.0037045   3.606 0.000311 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 812.84  on 616  degrees of freedom
Residual deviance: 785.65  on 614  degrees of freedom
AIC: 791.65

Number of Fisher Scoring iterations: 4


In [5]:
# Let's confirm this distrubtion by loading a different data set,
# namely the Intercontinental Dictionary Series:

# Download the current WALS data set to the cwd:
download.file("https://cdstar.shh.mpg.de/bitstreams/EAEA0-9C1A-66E2-D0B3-0/ids_dataset.cldf.zip",
              destfile = "ids_dataset.cldf.zip")

# Read language and value information from the ZIP file:
languages.ids <- read.csv(unz("ids_dataset.cldf.zip", "languages.csv"),
                      header=TRUE, sep=",")
parameters.ids <- read.csv(unz("ids_dataset.cldf.zip", "parameters.csv"),
                   header=TRUE, sep=",")
forms.ids <- read.csv(unz("ids_dataset.cldf.zip", "forms.csv"),
                           header=TRUE, sep=",", encoding="UTF-8")

meaning.hand <- "4-330"
meaning.arm <- "4-310"

hand.filtered <- forms.ids[forms.ids$Parameter_ID == meaning.hand,]
hand.filtered <- droplevels(hand.filtered)

arm.filtered <- forms.ids[forms.ids$Parameter_ID == meaning.arm,]
arm.filtered <- droplevels(arm.filtered)

head(arm.filtered)
head(hand.filtered)

,ID,Language_ID,Parameter_ID,Form,Segments,Comment,Source,Contribution_ID
231163,4-310-325-1,325,4-310,лыкьа,NA,NA,NA,325
231164,4-310-325-3,325,4-310,геша,NA,NA,NA,325
231165,4-310-319-1,319,4-310,къух,NA,NA,NA,319
231166,4-310-319-3,319,4-310,арм,NA,NA,NA,319
231167,4-310-320-1,320,4-310,квер,NA,NA,NA,320
231168,4-310-321-1,321,4-310,квер,NA,NA,NA,321


,ID,Language_ID,Parameter_ID,Form,Segments,Comment,Source,Contribution_ID
229840,4-330-325-1,325,4-330,ликьа,NA,NA,NA,325
229841,4-330-325-3,325,4-330,рацIцIалба,NA,NA,NA,325
229842,4-330-319-1,319,4-330,къух,NA,NA,NA,319
229843,4-330-319-3,319,4-330,арм,NA,NA,NA,319
229844,4-330-320-1,320,4-330,квер,NA,NA,NA,320
229845,4-330-321-1,321,4-330,квер,NA,NA,NA,321


In [6]:
# Prepare this for comparing it to WALS:
merged.hand.arm <- merge(hand.filtered, arm.filtered, by = "Language_ID", suffixes = c('.hand', '.arm'))
same.lexeme <- merged.hand.arm[as.character(merged.hand.arm$Form.hand) == as.character(merged.hand.arm$Form.arm), ]
colnames(languages.ids)[1] <- "Language_ID"

# Merge to get easy access to the Glottocode:
merge.for.comparison <- merge(same.lexeme, languages.ids, by = "Language_ID")

# Compare to the WALS feature selection from above:
overlap <- merged[merged$Glottocode %in% merge.for.comparison$Glottocode,]

# There is an overlap of 24 entries for the same/same entries in both databases:
dim(overlap)

[1] 24 14

In [7]:
# There are 8 languoids that have a "Different" value for the arm/hand colexification
# but have the same lexeme in IDS:
differences <- subset(overlap, Value == "Different")
dim(differences)

[1]  8 14

In [8]:
differences

,Language_ID,Name,Macroarea,Latitude,Longitude,Glottocode,ISO639P3code,ID,Parameter_ID,Value,Code_ID,Comment,Source,Contribution_ID
160,emb,Emberá (Northern),NA,6.833333,-77.16667,nort2972,emp,129A-emb,129A,Different,129A-2,NA,Huber-and-Reed-1992,129
189,grk,Greek (Modern),NA,39.000000,22.00000,mode1248,ell,129A-grk,129A,Different,129A-2,NA,Bergman-1968,129
223,iri,Irish,NA,53.000000,-8.00000,iris1253,gle,129A-iri,129A,Different,129A-2,NA,Siochfhradha-1958,129
305,mca,Maca,NA,-25.000000,-57.50000,maca1260,mca,129A-mca,129A,Different,129A-2,NA,Hunt-1915,129
311,mcv,Mocoví,NA,-28.000000,-60.50000,moco1246,moc,129A-mcv,129A,Different,129A-2,NA,Flury-1951,129
439,pol,Polish,NA,52.000000,20.00000,poli1260,pol,129A-pol,129A,Different,129A-2,NA,Bulas-and-Whitfield-1983,129
459,rap,Rapanui,NA,-27.116667,-109.36667,rapa1244,rap,129A-rap,129A,Different,129A-2,NA,Fuentes-1960,129
527,tob,Toba,NA,-26.500000,-59.00000,toba1269,tob,129A-tob,129A,Different,129A-2,NA,Buckwalter-1980,129
